In [1]:
import pandas as pd
import random 
import itertools
import json

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, join_stocks_crypto, run_kmeans_dtw, dtw_matrix_calc, run_min_variance

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
df_all_stocks = pd.read_csv('stocks_data.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')

joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'crypto_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates

In [3]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [4]:
#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [5]:
# for i in range(1,20):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets) 
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)

In [6]:
#run_kmeans_dtw(joined_df, n_clus=3)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

dtw_matrix = dtw_matrix_calc(joined_df) #calculates based on the pct change table

model = AgglomerativeClustering(
    n_clusters=3,          
    metric='precomputed',     
    linkage='average'         # average seems the best so far
)

labels = model.fit_predict(dtw_matrix)

cluster_dict = {ticker: int(label) for ticker, label in zip(joined_df.columns, labels)}

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
#K-Shape


In [ ]:
#Compare based on Silhouette Score and Elbow method
from sklearn.metrics import silhouette_score
score = silhouette_score(X, labels)